In [19]:
%pip install openai
%pip install python-dotenv


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [260]:
import json
import random
import openai
import os
import re
from dotenv import load_dotenv


In [261]:
def is_valid_number_format(value):
    pattern = r'^-?\d+(\.\d+)?$'
    return bool(re.match(pattern, value))


In [212]:
word_list = []
def load_word_list():
    with open("words_alpha.txt", "r") as file:
        for line in file:
            word_list.append(line.strip())
    return word_list

word_list = load_word_list()



In [20]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if OPENAI_API_KEY is None:
    raise ValueError("The OPENAI_API_KEY environment variable is not set.")


RANDOM_SEED = 42

In [21]:
import openai
import json
import random

class LanguageAgent:
    def __init__(self, model_name, strategy_agent, api_key):
        self.model_name = model_name
        self.client = openai.OpenAI(api_key=api_key)
        self.strategy_agent = strategy_agent
        self.past_guesses = []
        self.first_guess = True

    def build_context_message(self, word_list, constraints, word_length):
        constraints_serializable = {
            'green': constraints['green'],
            'yellow': list(constraints['yellow']),
            'grey': list(constraints['grey'])
        }
        context = f"""
        ### Wordle Context:
        - **Word List**: {', '.join(word_list)}
        - **Word Length**: {word_length}
        - **Constraints**: {json.dumps(constraints_serializable)}
        - **Previous Guesses**: {', '.join(self.past_guesses) if self.past_guesses else "None"}
        """
        return context

    def suggest_word(self, word_list, constraints, word_length):
        context_message = self.build_context_message(word_list, constraints, word_length)

        prompt = f"""
            You are a Wordle-solving assistant.
            Your task is to suggest the next best word based on the context below.
            
            ### Rules:
            1. The suggested word must match all constraints:
            - **Green**: Correct letter in the correct position.
            - **Yellow**: Correct letter in the wrong position.
            - **Grey**: Letter is not in the word.
            2. Prioritize words that maximize information gain.
            3. The word must be of length {word_length}.
            4. The word must be in the word list.
            5. I Just want the word, nothing else.
            
            Use the context provided to generate only the best word, with no extra text.
            
            {context_message}
        """

        try:
            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=[{"role": "system", "content": prompt}]
            )

            if response.choices:
                response_content = response.choices[0].message.content.strip()
                # Ensure only the word is returned
                word_suggestion = response_content.splitlines()[0]  # Get the first line
                self.past_guesses.append(word_suggestion)

                score_threshold = 0.8

                score = self.strategy_agent.evaluate_suggestion(
                    word_suggestion, constraints, self.past_guesses, word_length, score_threshold, context_message)
                print(f"Score: {score}")

                if (is_valid_number_format(score)):
                    return word_suggestion
                else:
                    return score
            else:
                raise ValueError("No choices returned from the API.")

        except Exception as e:
            print(f"An error occurred: {e}")
            return None

In [22]:
class StrategyAgent:
    def __init__(self, model_name, api_key):
        self.model_name = model_name
        self.client = openai.OpenAI(api_key=api_key)

    def evaluate_suggestion(self, suggestion, constraints, past_guesses, word_length, threshold, context_message):
        """
        Uses GPT to evaluate whether a suggested Wordle word is valid based on given constraints.
        Returns a score between 0 (bad) and 1 (perfect).
        """
        # Convert constraints to a serializable format
        constraints_serializable = {
            'green': constraints.get('green', {}),
            'yellow': list(constraints.get('yellow', [])),
            'grey': list(constraints.get('grey', []))
        }
        
        # Create a unified prompt that contains all necessary context
        prompt = f"""
            You are an AI evaluator for Wordle strategies. Your task is to assess a suggested word based on the following criteria:
            - **Word Length**: {word_length}
            - **Previous Guesses**: {', '.join(past_guesses)}
            - **Constraints**:
              - **Green**: Letters in the correct positions.
              - **Yellow**: Letters in the word but in the wrong positions.
              - **Grey**: Letters that do not appear in the word at all.

            Evaluate the following suggestion: {suggestion}.

            Take into account the following context of the wordle game: 
            
            {context_message}

            If the score you would give is less than the threshold of {threshold}, please provide a better suggestion instead of the score. If the score is greater than or equal to the threshold, return the score as a number.
            I dont want any extra text, just the score or the better suggestion nothing else. Dont even put a dot at the end of the score or the better suggestion. dont precise that it is a score or a better suggestion.
        """

        try:
            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": f"Suggestion: {suggestion}"},
                    {"role": "user", "content": f"Constraints: {json.dumps(constraints_serializable)}"}
                ]
            )

            score_str = response.choices[0].message.content.strip()
            print("Score_str is : ", score_str)
            return score_str
        except Exception as e:
            print(f"Error evaluating suggestion: {e}")
            score = 0.5

        return score

In [23]:
class MonitorAgent:
    def __init__(self, language_agent, word_length):
        self.language_agent = language_agent
        self.constraints = {
            'green': {},
            'yellow': set(),
            'grey': set()
        }
        self.word_length = word_length

    def generate_word_to_guess(self, word_list, expected_length):
        random.seed(RANDOM_SEED)
        valid_words = [word for word in word_list if len(word) == expected_length]  # Define expected_length appropriately
        if valid_words:
            word = random.choice(valid_words)
            print(f"Word to guess: {word}")
            return word
        else:
            print("No valid words found for the given length.")
            return None 
    
    def update_constraints(self, guess, feedback):
        for i, (letter, result) in enumerate(zip(guess, feedback)):
            if result == 'green':
                self.constraints['green'][i] = letter
            elif result == 'yellow':
                self.constraints['yellow'].add(letter)
            elif result == 'grey':
                self.constraints['grey'].add(letter)
    
    def generate_feedback(self, guess, word):
        feedback = []
        if len(guess) != len(word):
            print(f"Error: Guess '{guess}' and word '{word}' must be of the same length.")
            return None
        
        for i, letter in enumerate(guess):
            if letter == word[i]:
                feedback.append('green')
            elif letter in word:
                feedback.append('yellow')
            else:
                feedback.append('grey')
        return feedback

    def get_word_list(self, word_list):
        valid_words = []
        for word in word_list:
            if len(word) == self.word_length:
                valid_words.append(word)
        return valid_words
    
    def solve_wordle(self, word_list):
        word_list = self.get_word_list(word_list)
        print("Starting Wordle solver...")
        word = self.generate_word_to_guess(word_list, self.word_length)
        print(f"Word to guess: {word}")
        for i in range(len(word)+10):
            suggestion = self.language_agent.suggest_word(word_list, self.constraints, len(word))
            print(f"Suggestion: {suggestion}")
            feedback = self.generate_feedback(suggestion, word)
            if feedback is None:
                print("Feedback is None")
                continue
            self.update_constraints(suggestion, feedback)
            if all(result == 'green' for result in feedback):
                print("--------------------------------")
                print("Feedback: ", feedback)
                print(f"Found the word: {suggestion}")
                print(f"Number of guesses: {i}")
                return suggestion
            else:
                print(f"Current constraints: {self.constraints}")
                print(f"Current guess: {suggestion}")
                print(f"Feedback: {feedback}")
                print("--------------------------------")
        print("Failed to find the word")
        return None


    


In [25]:
strategy_agent:StrategyAgent = StrategyAgent(model_name="gpt-4o-mini", api_key=OPENAI_API_KEY)
language_agent:LanguageAgent = LanguageAgent(model_name="gpt-4o-mini", strategy_agent=strategy_agent, api_key=OPENAI_API_KEY)
agent:MonitorAgent = MonitorAgent(language_agent, 5)


In [27]:
agent.solve_wordle(word_list)

Starting Wordle solver...
Word to guess: peppy
Word to guess: peppy
Score_str is :  peppy
Score: peppy
Suggestion: peppy
--------------------------------
Feedback:  ['green', 'green', 'green', 'green', 'green']
Found the word: peppy
Number of guesses: 0


'peppy'